In [1]:
import os

desired_path = "C:\\Users\\kosti\\OneDrive - Εθνικό Μετσόβιο Πολυτεχνείο\\ΣΗΜΜΥ_2024_2025\\Αναγνώριση Προτύπων\\1ο Εργαστήριο\\torch_tutorials"
os.chdir(desired_path)
cwd = os.getcwd()
print(cwd)

C:\Users\kosti\OneDrive - Εθνικό Μετσόβιο Πολυτεχνείο\ΣΗΜΜΥ_2024_2025\Αναγνώριση Προτύπων\1ο Εργαστήριο\torch_tutorials


In [4]:
from io import open 
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('./data/names/*.txt'))

['./data/names\\Arabic.txt', './data/names\\Chinese.txt', './data/names\\Czech.txt', './data/names\\Dutch.txt', './data/names\\English.txt', './data/names\\French.txt', './data/names\\German.txt', './data/names\\Greek.txt', './data/names\\Irish.txt', './data/names\\Italian.txt', './data/names\\Japanese.txt', './data/names\\Korean.txt', './data/names\\Polish.txt', './data/names\\Portuguese.txt', './data/names\\Russian.txt', './data/names\\Scottish.txt', './data/names\\Spanish.txt', './data/names\\Vietnamese.txt']


In [5]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

Slusarski


In [7]:
category_lines = {}
all_categories = []

def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('./data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines
    
n_categories = len(all_categories)

In [8]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [9]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [11]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.U = nn.Linear(input_size, hidden_size, bias=True)
        self.W= nn.Linear(hidden_size, hidden_size, bias=True)
        self.V = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        a = self.U(input) + self.W(hidden)
        hidden = F.tanh(a)
        output = self.V(hidden)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)


In [12]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input, hidden)

In [13]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.8839, -2.9062, -2.9500, -2.7771, -2.9307, -2.9703, -2.8588, -2.8797,
         -2.9757, -2.7710, -2.9078, -2.7282, -2.9240, -2.8237, -2.9245, -2.9096,
         -2.9768, -2.9777]], grad_fn=<LogSoftmaxBackward0>)


In [15]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    print(top_n, top_i)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

tensor([[-2.7282]], grad_fn=<TopkBackward0>) tensor([[11]])
('Korean', 11)


In [16]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Irish / line = Rhys
category = English / line = Main
category = Scottish / line = Hunter
category = Portuguese / line = Ribeiro
category = Arabic / line = Shalhoub
category = Vietnamese / line = Vuu
category = French / line = Leveque
category = Italian / line = Goretti
category = Dutch / line = Pander
category = Vietnamese / line = Huynh


In [19]:
criterion = nn.NLLLoss() # -logits[y]

In [20]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [21]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print ``iter`` number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

tensor([[-2.0061]], grad_fn=<TopkBackward0>) tensor([[14]])
5000 5% (0m 9s) 2.0061 Shalashilin / Russian ✓
tensor([[-0.7612]], grad_fn=<TopkBackward0>) tensor([[0]])
10000 10% (0m 18s) 0.7612 Safar / Arabic ✓
tensor([[-1.3413]], grad_fn=<TopkBackward0>) tensor([[8]])
15000 15% (0m 27s) 1.8114 Tahan / Irish ✗ (Arabic)
tensor([[-0.0632]], grad_fn=<TopkBackward0>) tensor([[7]])
20000 20% (0m 37s) 0.0632 Garofalis / Greek ✓
tensor([[-0.8592]], grad_fn=<TopkBackward0>) tensor([[9]])
25000 25% (0m 46s) 2.3682 Elizondo / Italian ✗ (Spanish)
tensor([[-0.1765]], grad_fn=<TopkBackward0>) tensor([[0]])
30000 30% (0m 55s) 0.1765 Daher / Arabic ✓
tensor([[-1.7392]], grad_fn=<TopkBackward0>) tensor([[6]])
35000 35% (1m 5s) 2.3607 Patrick / German ✗ (Irish)
tensor([[-0.5846]], grad_fn=<TopkBackward0>) tensor([[2]])
40000 40% (1m 16s) 0.5846 Pavlicka / Czech ✓
tensor([[-0.0122]], grad_fn=<TopkBackward0>) tensor([[14]])
45000 45% (1m 26s) 0.0122 Lezhepekov / Russian ✓
tensor([[-1.1352]], grad_fn=<TopkB

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)